# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [7]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [14]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','Metros_total':'metros_total','m_total':'metros_total','m_construido':'metros_construido','Metros_construido':'metros_construido','m2_contruido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','Precio_M2_Construido':'precio_m2_construido','precio_m2_contruido':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [15]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [16]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [17]:
df_total.columns

Index(['id', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [18]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21558 entries, 0 to 21557
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    21558 non-null  int64  
 1   categoria             21558 non-null  object 
 2   propiedad             21558 non-null  object 
 3   precio                21558 non-null  float64
 4   metros_total          21558 non-null  int64  
 5   metros_construido     21558 non-null  int64  
 6   precio_m2_construido  21558 non-null  float64
 7   estacionamiento       21558 non-null  int64  
 8   recamaras             21558 non-null  int64  
 9   baño                  21558 non-null  int64  
 10  medio_baño            21558 non-null  int64  
 11  baño_total            21558 non-null  float64
dtypes: float64(3), int64(7), object(2)
memory usage: 2.0+ MB


In [19]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

df_total = df_total.drop(columns=['categoria'])
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

In [20]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [21]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)



print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)


# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    3.0
L                      3.0        4.0    3.0
S                      2.0        4.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      3  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    3.0
L                      3.0        4.0    3.0
S                      2.0        4.0    3.0
general                2.0        3.0    3.0


# Casa Mediana

In [22]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    1.0
E                      2.0        3.0    3.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    1.0
E                      2.0        3.0    3.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
general                2.0        3.0    2.0


# Precio Mercado

In [24]:
df_total = df_total[df_total['precio'] != 0]

In [25]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3255000.0,3999000.0,3.602734e+06,3600000.00
1,B,2500002.0,3242320.0,2.881888e+06,2890000.00
2,C,1750050.0,2499000.0,2.115238e+06,2140000.00
3,D,1005000.0,1749605.0,1.394899e+06,1400000.00
4,E,4800.0,90000000.0,1.094976e+06,999999.99
5,L,12100000.0,21000000.0,1.616734e+07,15500000.00
6,S,4020090.0,11900000.0,5.926903e+06,5350000.00


In [26]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,255,000.00
1     $2,500,002.00
2     $1,750,050.00
3     $1,005,000.00
4         $4,800.00
5    $12,100,000.00
6     $4,020,090.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,999,000.00
1     $3,242,320.00
2     $2,499,000.00
3     $1,749,605.00
4    $90,000,000.00
5    $21,000,000.00
6    $11,900,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripci

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,255,000.00","$3,999,000.00","$3,602,733.51","$3,600,000.00"
1,B,"$2,500,002.00","$3,242,320.00","$2,881,888.21","$2,890,000.00"
2,C,"$1,750,050.00","$2,499,000.00","$2,115,238.38","$2,140,000.00"
3,D,"$1,005,000.00","$1,749,605.00","$1,394,898.97","$1,400,000.00"
4,E,"$4,800.00","$90,000,000.00","$1,094,975.62","$999,999.99"
5,L,"$12,100,000.00","$21,000,000.00","$16,167,342.34","$15,500,000.00"
6,S,"$4,020,090.00","$11,900,000.00","$5,926,902.77","$5,350,000.00"


In [27]:
todo_descripcion = df_total['precio'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $4,800.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $90,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in

,Mínimo,Máximo,Promedio,Mediana,segmento
precio,"$4,800.00","$90,000,000.00","$2,190,466.12","$999,999.99",general


In [28]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [29]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,255,000.00","$3,999,000.00","$3,602,733.51","$3,600,000.00"
1,B,"$2,500,002.00","$3,242,320.00","$2,881,888.21","$2,890,000.00"
2,C,"$1,750,050.00","$2,499,000.00","$2,115,238.38","$2,140,000.00"
3,D,"$1,005,000.00","$1,749,605.00","$1,394,898.97","$1,400,000.00"
4,E,"$4,800.00","$90,000,000.00","$1,094,975.62","$999,999.99"
5,L,"$12,100,000.00","$21,000,000.00","$16,167,342.34","$15,500,000.00"
6,S,"$4,020,090.00","$11,900,000.00","$5,926,902.77","$5,350,000.00"
7,general,"$4,800.00","$90,000,000.00","$2,190,466.12","$999,999.99"


# M2 Terreno

In [32]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[df_total_terreno['metros_total'] != 1]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\3460250308.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     75.00 m²
1     62.00 m²
2     44.00 m²
3     15.00 m²
4     30.00 m²
5    245.00 m²
6     85.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\3460250308.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     1,077.00 m²
1    15,794.00 m²
2    14,079.00 m²
3     8,315.00 m²
4    16,667.00 m²
5    24,569.00 m²
6    32,240.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).a

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,"1,077.00 m²",163.45 m²,144.00 m²
1,B,62.00 m²,"15,794.00 m²",171.47 m²,129.00 m²
2,C,44.00 m²,"14,079.00 m²",149.37 m²,120.00 m²
3,D,15.00 m²,"8,315.00 m²",121.92 m²,105.00 m²
4,E,30.00 m²,"16,667.00 m²",162.99 m²,126.00 m²
5,L,245.00 m²,"24,569.00 m²",774.62 m²,500.00 m²
6,S,85.00 m²,"32,240.00 m²",260.96 m²,200.00 m²


In [34]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[df_total_terreno['metros_total'] != 1]

todo_descripcion = df_total_terreno[df_total_terreno['metros_total'] != 0]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\628811919.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    15.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\628811919.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    32,240.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\628811919.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,15.00 m²,"32,240.00 m²",176.38 m²,128.00 m²,general


In [35]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [36]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,"1,077.00 m²",163.45 m²,144.00 m²
1,B,62.00 m²,"15,794.00 m²",171.47 m²,129.00 m²
2,C,44.00 m²,"14,079.00 m²",149.37 m²,120.00 m²
3,D,15.00 m²,"8,315.00 m²",121.92 m²,105.00 m²
4,E,30.00 m²,"16,667.00 m²",162.99 m²,126.00 m²
5,L,245.00 m²,"24,569.00 m²",774.62 m²,500.00 m²
6,S,85.00 m²,"32,240.00 m²",260.96 m²,200.00 m²
7,general,15.00 m²,"32,240.00 m²",176.38 m²,128.00 m²


# M2 Construcción

In [43]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     75.00 m²
1     62.00 m²
2     44.00 m²
3     45.00 m²
4     25.00 m²
5    245.00 m²
6     30.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\2035657892.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       460.00 m²
1    29,743.00 m²
2       640.00 m²
3       472.00 m²
4    10,000.00 m²
5     1,000.00 m²
6     1,465.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).a

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,460.00 m²,216.10 m²,222.00 m²
1,B,62.00 m²,"29,743.00 m²",230.52 m²,200.00 m²
2,C,44.00 m²,640.00 m²,159.53 m²,150.00 m²
3,D,45.00 m²,472.00 m²,119.72 m²,114.00 m²
4,E,25.00 m²,"10,000.00 m²",200.61 m²,170.00 m²
5,L,245.00 m²,"1,000.00 m²",567.30 m²,539.00 m²
6,S,30.00 m²,"1,465.00 m²",300.66 m²,280.00 m²


In [44]:
df_total_terreno = df_total[df_total['metros_construido'] > 5]

todo_descripcion = df_total_terreno['metros_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\1652338033.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    25.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\1652338033.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    29,743.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\1652338033.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and 

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_construido,25.00 m²,"29,743.00 m²",205.21 m²,175.00 m²,general


In [45]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [46]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,460.00 m²,216.10 m²,222.00 m²
1,B,62.00 m²,"29,743.00 m²",230.52 m²,200.00 m²
2,C,44.00 m²,640.00 m²,159.53 m²,150.00 m²
3,D,45.00 m²,472.00 m²,119.72 m²,114.00 m²
4,E,25.00 m²,"10,000.00 m²",200.61 m²,170.00 m²
5,L,245.00 m²,"1,000.00 m²",567.30 m²,539.00 m²
6,S,30.00 m²,"1,465.00 m²",300.66 m²,280.00 m²
7,general,25.00 m²,"29,743.00 m²",205.21 m²,175.00 m²


# Pecio M2 Construcción

In [47]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\3926392529.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $8,043.48
1       $107.25
2     $3,358.64
3     $2,400.66
4        $19.28
5    $17,375.00
6     $2,779.52
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\3926392529.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $47,500.00
1     $44,692.00
2     $42,771.66
3     $38,000.00
4    $295,800.00
5     $61,224.49
6    $238,333.33
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$8,043.48","$47,500.00","$18,091.71","$16,237.29"
1,B,$107.25,"$44,692.00","$16,086.00","$14,950.00"
2,C,"$3,358.64","$42,771.66","$14,335.25","$14,000.01"
3,D,"$2,400.66","$38,000.00","$12,598.79","$12,234.04"
4,E,$19.28,"$295,800.00","$14,557.42","$13,750.00"
5,L,"$17,375.00","$61,224.49","$30,200.68","$27,777.78"
6,S,"$2,779.52","$238,333.33","$21,695.69","$19,218.80"


In [48]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $19.28
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $295,800.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_4136\756331146.py:13: FutureWarning: Setting an item of incompatible dtype is deprecate

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,$19.28,"$295,800.00","$15,529.96","$14,344.26",general


In [49]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [50]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$8,043.48","$47,500.00","$18,091.71","$16,237.29"
1,B,$107.25,"$44,692.00","$16,086.00","$14,950.00"
2,C,"$3,358.64","$42,771.66","$14,335.25","$14,000.01"
3,D,"$2,400.66","$38,000.00","$12,598.79","$12,234.04"
4,E,$19.28,"$295,800.00","$14,557.42","$13,750.00"
5,L,"$17,375.00","$61,224.49","$30,200.68","$27,777.78"
6,S,"$2,779.52","$238,333.33","$21,695.69","$19,218.80"
7,general,$19.28,"$295,800.00","$15,529.96","$14,344.26"
